In [2]:
:dep aoc2023 = { path = "../../" }
:dep anyhow = { version = "1.0.75" }

In [53]:
use aoc2023::utils::grid::{Grid, Coord, Direction, Neighbor};
use aoc2023::utils::util;
use std::collections::{HashMap, HashSet, VecDeque};

In [11]:
let lines = util::lines_in("./input");

In [5]:
use aoc2023::utils::util::ToGrid;

In [6]:
lines.len()

23

In [7]:
let grid = lines.to_grid();

In [8]:
let start = Coord::new(
    0,
    grid.row(0)
        .unwrap()
        .iter()
        .position(|val| **val == '.')
        .unwrap(),
);

In [9]:
start

Coord { p: 0, q: 1 }

In [35]:
fn free_neighbors(grid: &Grid<char>, from: Coord) -> Vec<Neighbor<&char>> {
    grid.neighbors(&from)
        .into_iter()
        .filter(|neighbor| *neighbor.cell.val != '#')
        .collect::<Vec<Neighbor<&char>>>()
}

fn valid_neighbors_part2(grid: &Grid<char>, from: Coord, from_dir: Direction) -> Vec<Neighbor<&char>> {
    grid.neighbors(&from)
        .into_iter()
        .filter(|neighbor| *neighbor.cell.val != '#' && neighbor.dir != from_dir)
        .collect::<Vec<Neighbor<&char>>>()
}

fn next(grid: &Grid<char>, from: Coord, from_dir: Direction) -> Option<(Coord, usize)> {
    let mut next_neighbors = valid_neighbors_part2(grid, from, from_dir);
    let mut curr = from;
    let mut dist = 0;
    if next_neighbors.len() == 0 {
        return None;
    }

    while next_neighbors.len() == 1 {
        let neighbor = next_neighbors.pop().unwrap();
        curr = neighbor.cell.coord;
        dist += 1;
        next_neighbors = valid_neighbors_part2(grid, curr, neighbor.dir.opposite());

        if next_neighbors.len() == 0 {
            return None;
        }
    }
    Some((curr, dist))
}

In [36]:
#[derive(Debug)]
struct Edge {
    nodes: (Coord, Coord),
    dist: usize,
}

impl Edge {
    fn new(nodes: (Coord, Coord), dist: usize) -> Edge {
        Edge { nodes, dist }
    }
}

impl PartialEq for Edge {
    fn eq(&self, other: &Self) -> bool {
        (self.nodes.0 == other.nodes.0 && self.nodes.1 == other.nodes.1)
            || (self.nodes.0 == other.nodes.1 && self.nodes.1 == other.nodes.0)
    }
}

impl Eq for Edge {}

In [37]:
let end = Coord::new(
    grid.m - 1,
    grid.row(grid.m - 1)
        .unwrap()
        .iter()
        .position(|val| **val == '.')
        .unwrap(),
);

In [38]:
end

Coord { p: 22, q: 21 }

In [60]:
let mut edges = vec![];
let (start_edge_other, dist) = next(&grid, start, Direction::Top).unwrap();
edges.push(Edge::new((start, start_edge_other), dist));

In [61]:
    let (end_edge_other, dist) = next(&grid, end, Direction::Bottom).unwrap();
    edges.push(Edge::new((end, end_edge_other), dist));

In [62]:
edges

[Edge { nodes: (Coord { p: 0, q: 1 }, Coord { p: 5, q: 3 }), dist: 15 }, Edge { nodes: (Coord { p: 22, q: 21 }, Coord { p: 19, q: 19 }), dist: 5 }]

In [63]:
let mut queue = VecDeque::new();
queue.push_back(start_edge_other);
while !queue.is_empty() {
    let jnode = queue.pop_front().unwrap();
    let neighbors = free_neighbors(&grid, jnode);

    for neighbor in neighbors {
        match next(&grid, neighbor.cell.coord, neighbor.dir.opposite()) {
            Some((jnode_other, dist)) => {
                let edge = Edge::new((jnode, jnode_other), dist);
                if !edges.contains(&edge) {
                    edges.push(edge);
                    queue.push_back(jnode_other);
                }
            }
            None => {}
        }
    }
}

()

In [64]:
edges.len()

12

In [65]:
edges

[Edge { nodes: (Coord { p: 0, q: 1 }, Coord { p: 5, q: 3 }), dist: 15 }, Edge { nodes: (Coord { p: 22, q: 21 }, Coord { p: 19, q: 19 }), dist: 5 }, Edge { nodes: (Coord { p: 5, q: 3 }, Coord { p: 3, q: 11 }), dist: 21 }, Edge { nodes: (Coord { p: 5, q: 3 }, Coord { p: 13, q: 5 }), dist: 21 }, Edge { nodes: (Coord { p: 3, q: 11 }, Coord { p: 11, q: 21 }), dist: 29 }, Edge { nodes: (Coord { p: 3, q: 11 }, Coord { p: 13, q: 13 }), dist: 23 }, Edge { nodes: (Coord { p: 13, q: 5 }, Coord { p: 13, q: 13 }), dist: 11 }, Edge { nodes: (Coord { p: 13, q: 5 }, Coord { p: 19, q: 13 }), dist: 37 }, Edge { nodes: (Coord { p: 11, q: 21 }, Coord { p: 13, q: 13 }), dist: 17 }, Edge { nodes: (Coord { p: 11, q: 21 }, Coord { p: 19, q: 19 }), dist: 9 }, Edge { nodes: (Coord { p: 13, q: 13 }, Coord { p: 19, q: 13 }), dist: 9 }, Edge { nodes: (Coord { p: 19, q: 13 }, Coord { p: 19, q: 19 }), dist: 9 }]

In [66]:
fn add_adj_node(
    adj: &mut HashMap<Coord, Vec<(Coord, usize)>>,
    from: Coord,
    to: Coord,
    dist: usize,
) {
    match adj.get_mut(&from) {
        Some(to_vec) => to_vec.push((to, dist)),
        None => {
            adj.insert(from, vec![(to, dist)]);
        }
    }
}

let mut adj = HashMap::new();
for edge in edges {
    let (from, to) = edge.nodes;
    add_adj_node(&mut adj, from, to, edge.dist);
    add_adj_node(&mut adj, to, from, edge.dist);
}

()

In [67]:
adj

{Coord { p: 5, q: 3 }: [(Coord { p: 0, q: 1 }, 15), (Coord { p: 3, q: 11 }, 21), (Coord { p: 13, q: 5 }, 21)], Coord { p: 3, q: 11 }: [(Coord { p: 5, q: 3 }, 21), (Coord { p: 11, q: 21 }, 29), (Coord { p: 13, q: 13 }, 23)], Coord { p: 19, q: 13 }: [(Coord { p: 13, q: 5 }, 37), (Coord { p: 13, q: 13 }, 9), (Coord { p: 19, q: 19 }, 9)], Coord { p: 22, q: 21 }: [(Coord { p: 19, q: 19 }, 5)], Coord { p: 13, q: 5 }: [(Coord { p: 5, q: 3 }, 21), (Coord { p: 13, q: 13 }, 11), (Coord { p: 19, q: 13 }, 37)], Coord { p: 11, q: 21 }: [(Coord { p: 3, q: 11 }, 29), (Coord { p: 13, q: 13 }, 17), (Coord { p: 19, q: 19 }, 9)], Coord { p: 13, q: 13 }: [(Coord { p: 3, q: 11 }, 23), (Coord { p: 13, q: 5 }, 11), (Coord { p: 11, q: 21 }, 17), (Coord { p: 19, q: 13 }, 9)], Coord { p: 19, q: 19 }: [(Coord { p: 22, q: 21 }, 5), (Coord { p: 11, q: 21 }, 9), (Coord { p: 19, q: 13 }, 9)], Coord { p: 0, q: 1 }: [(Coord { p: 5, q: 3 }, 15)]}